#### Import Lib

In [57]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import KFold, train_test_split, GridSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#### Đọc dữ liệu

In [58]:
df_real = pd.read_csv('../Data/input_data.csv')
df = df_real.copy()

In [59]:
# # Hàm loại bỏ giá trị ngoại lai
# def remove_outliers(df, columns=None):
#     """
#     Loại bỏ giá trị ngoại lai khỏi các cột được chỉ định trong DataFrame.
#     - df: DataFrame đầu vào
#     - columns: Danh sách các cột cần kiểm tra (nếu None, sẽ kiểm tra tất cả các cột số)
#     """
#     if columns is None:
#         columns = df.select_dtypes(include=['float', 'int']).columns  # Chỉ lấy các cột số

#     for col in columns:
#         # Tính IQR (Interquartile Range)
#         Q1 = df[col].quantile(0.25)
#         Q3 = df[col].quantile(0.75)
#         IQR = Q3 - Q1
#         # Xác định phạm vi hợp lệ
#         lower_bound = Q1 - 1.5 * IQR
#         upper_bound = Q3 + 1.5 * IQR
#         # Loại bỏ ngoại lai
#         df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
#     return df

# # Áp dụng hàm
# df = remove_outliers(df)

In [60]:
df.columns

Index(['Brand', 'Processor_Name', 'Processor_Brand', 'RAM', 'RAM_TYPE',
       'Display_type', 'GPU', 'GPU_Brand', 'SSD', 'OS', 'VRAM', 'Price_VND'],
      dtype='object')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Brand            3916 non-null   object
 1   Processor_Name   3916 non-null   object
 2   Processor_Brand  3916 non-null   object
 3   RAM              3916 non-null   int64 
 4   RAM_TYPE         3916 non-null   object
 5   Display_type     3916 non-null   object
 6   GPU              3916 non-null   object
 7   GPU_Brand        3916 non-null   object
 8   SSD              3916 non-null   int64 
 9   OS               3916 non-null   object
 10  VRAM             3916 non-null   int64 
 11  Price_VND        3916 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 367.3+ KB


#### Train - Test Split

In [62]:
# Giả sử dữ liệu của bạn có tên là df
X = df.drop('Price_VND', axis=1)  # Các biến độc lập
y = df['Price_VND']  # Biến mục tiêu

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
# Xác định biến số và biến phân loại
# Xác định các biến số và biến phân loại
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

#### Tạo Pipeline tiền xử lý

In [64]:
# Tạo pipeline cho tiền xử lý
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Kết hợp các bước tiền xử lý vào ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

#### Xây dựng Pipeline cho mô hình hồi quy

In [65]:
# Danh sách các mô hình hồi quy
models = {
    'XGBRegressor': XGBRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'KNeighborsRegressor': KNeighborsRegressor()
}

# Huấn luyện và đánh giá mô hình
results = {}

for model_name, model in models.items():
    # Tạo pipeline cho mô hình
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    
    # Huấn luyện mô hình
    pipeline.fit(X_train, y_train)
    
    # Dự đoán trên tập train và tập test
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # Đánh giá mô hình trên tập train
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Đánh giá mô hình trên tập test
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Lưu kết quả đánh giá
    results[model_name] = {
        'Train MAE': train_mae, 'Test MAE': test_mae,
        'Train MSE': train_mse, 'Test MSE': test_mse,
        'Train R2': train_r2,   'Test R2': test_r2
    }

# Hiển thị kết quả
results_df = pd.DataFrame(results).T
results_df


,Train MAE,Test MAE,Train MSE,Test MSE,Train R2,Test R2
XGBRegressor,2.385796e+06,3.702117e+06,1.300392e+13,4.114120e+13,0.945902,0.817971
DecisionTreeRegressor,9.370600e+05,4.361281e+06,5.282414e+12,5.685717e+13,0.978025,0.748436
RandomForestRegressor,1.864361e+06,3.797793e+06,9.741736e+12,4.522612e+13,0.959473,0.799897
LinearRegression,3.345550e+06,3.952811e+06,2.894091e+13,4.428429e+13,0.879603,0.804064
Ridge,3.529900e+06,3.989395e+06,3.065123e+13,4.357296e+13,0.872488,0.807212
KNeighborsRegressor,3.232276e+06,4.069476e+06,2.914730e+13,5.130707e+13,0.878744,0.772992


In [66]:
# # Danh sách các mô hình hồi quy
# models = {
#     'XGBRegressor': XGBRegressor(),
#     'DecisionTreeRegressor': DecisionTreeRegressor(),
#     'RandomForestRegressor': RandomForestRegressor(),
#     'LinearRegression': LinearRegression(),
#     'Ridge': Ridge(),
#     'KNeighborsRegressor': KNeighborsRegressor()
# }

# # Huấn luyện và đánh giá mô hình
# results = {}

# for model_name, model in models.items():
#     # Tạo pipeline cho mô hình
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                ('regressor', model)])
    
#     # Huấn luyện mô hình
#     pipeline.fit(X_train, y_train)
    
#     # Dự đoán trên tập kiểm tra
#     y_pred = pipeline.predict(X_test)
    
#     # Đánh giá mô hình
#     mae = mean_absolute_error(y_test, y_pred)
#     mse = mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)
    
#     # Lưu kết quả đánh giá
#     results[model_name] = {'MAE': mae, 'MSE': mse, 'R2': r2}

# # Hiển thị kết quả
# results_df = pd.DataFrame(results).T
# results_df


Kết quả:
- Mô hình tốt nhất: XGBRegressor có R² cao nhất (81.8%) và MAE thấp, làm cho nó trở thành mô hình tốt nhất trong bộ so sánh này.
- Mô hình kém nhất: KNeighborsRegressor có R² thấp nhất (78.4%) và MSE cao nhất, cho thấy nó không phù hợp với dữ liệu này so với các mô hình khác.

Hướng phát triển:
-  Tối ưu tham số cho mô hình XGBRegressor

#### Tối ưu tham số cho mô hình XGBRegressor

In [67]:
# Danh sách một số param cần lưu ý
xgb_params = {
    'n_estimators',        # Số lượng cây quyết định
    'learning_rate',       # Tốc độ học (eta)
    'max_depth',             # Độ sâu tối đa của cây
    'min_child_weight',      # Trọng số tối thiểu của mỗi nút lá
    'subsample',           # Tỷ lệ mẫu huấn luyện mỗi cây
    'colsample_bytree',    # Tỷ lệ đặc trưng được chọn để huấn luyện mỗi cây
    'gamma',                 # Mức độ phạt để ngừng tạo nhánh
    'scale_pos_weight',      # Tỷ lệ giữa các lớp trong bài toán mất cân bằng
    'objective',  # Mục tiêu hồi quy
    'eval_metric',      # Chỉ số đánh giá
    'tree_method',      # Phương pháp xây dựng cây
    'lambda',              # Regularization L2 (Ridge)
    'alpha'               # Regularization L1 (Lasso)
}


In [68]:
# Xây dựng Pipeline riêng cho mô hình XGBRegressor
XGB_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor())  # Mô hình XGBRegressor
    ]
)

# Huấn luyện lại mô hình với tham số mặc định
XGB_pipeline.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình với tham số mặc định
y_pred = XGB_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# In ra các kết quả đánh giá và tham số mặc định
print("Kết quả đánh giá với tham số mặc định:")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")
# XGB_pipeline.named_steps['regressor'].get_params()

# Tạo DataFrame từ y_test và y_pred
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})


# In ra các tham số có trong danh sách xgb_params
print("\nCác tham số của XGBRegressor trong danh sách cần lưu ý:")
xgb_model_params = XGB_pipeline.named_steps['regressor'].get_params()

for param in xgb_params:
    if param in xgb_model_params:
        print(f"{param}: {xgb_model_params[param]}")


Kết quả đánh giá với tham số mặc định:
MAE: 3702117.4776785714
MSE: 41141201332121.26
R²: 0.817970871925354

Các tham số của XGBRegressor trong danh sách cần lưu ý:
tree_method: None
n_estimators: None
colsample_bytree: None
learning_rate: None
objective: reg:squarederror
max_depth: None
subsample: None
eval_metric: None
scale_pos_weight: None
gamma: None
min_child_weight: None


In [ ]:
# Hàm dự đoán giá bán laptop
def predict_laptop_price(brand, processor_name, processor_brand, ram, ram_type, display_type, gpu, gpu_brand, ssd, os, vram):
    # Tạo DataFrame cho dữ liệu nhập
    input_data = pd.DataFrame({
        'Brand': [brand],
        'Processor_Name': [processor_name],
        'Processor_Brand': [processor_brand],
        'RAM': [ram],
        'RAM_TYPE': [ram_type],
        'Display_type': [display_type],
        'GPU': [gpu],
        'GPU_Brand': [gpu_brand],
        'SSD': [ssd],
        'OS': [os],
        'VRAM': [vram]
    })
    
    # Dự đoán giá bán
    predicted_price = XGB_pipeline.predict(input_data)
    return predicted_price[0]

# Ví dụ sử dụng hàm
predicted_price = predict_laptop_price(
    brand='Dell', processor_name='Intel Core i7 (13th Gen)', processor_brand='Intel', ram=32, ram_type='DDR5', 
    display_type='LED', gpu='Integrated Graphics', gpu_brand='Intel', ssd=1024, os='Windows 11', vram=0
)

print(f"Giá bán dự đoán của laptop là: {predicted_price} VND")

Giá bán dự đoán của laptop là: 65538712.0 VND


In [54]:
results_df

,Actual,Predicted
270,14637000,16533901.0
1972,21599700,17362346.0
802,16197000,17239934.0
2898,16027500,17164470.0
2846,17997000,15674817.0
...,...,...
2321,23339700,15332533.0
2694,34947000,32701512.0
1270,10497000,12711098.0
1132,35279700,35356932.0


In [55]:
# # Định nghĩa danh sách các tham số cần điều chỉnh
# # param_grid = {
# #     'regressor__n_estimators': [50, 100, 200],        # Số lượng cây quyết định
# #     'regressor__learning_rate': [0.01, 0.05, 0.1],    # Tốc độ học (eta)
# #     'regressor__max_depth': [3, 6, 10],                 # Độ sâu tối đa của cây
# #     'regressor__min_child_weight': [1, 5, 10],          # Trọng số tối thiểu của mỗi nút lá
# #     'regressor__subsample': [0.7, 0.8, 1.0],           # Tỷ lệ mẫu huấn luyện mỗi cây
# #     'regressor__colsample_bytree': [0.7, 0.8, 1.0],    # Tỷ lệ đặc trưng được chọn để huấn luyện mỗi cây
# #     'regressor__gamma': [0, 0.1, 0.2],                 # Mức độ phạt để ngừng tạo nhánh
# #     'regressor__scale_pos_weight': [1, 10],             # Tỷ lệ giữa các lớp trong bài toán mất cân bằng
# #     'regressor__tree_method': ['auto', 'hist'],         # Phương pháp xây dựng cây
# #     'regressor__lambda': [0.5, 1.0],                    # Regularization L2 (Ridge)
# #     'regressor__alpha': [0.0, 0.5],                     # Regularization L1 (Lasso)
# # }
# # param_grid = {
# #     'regressor__n_estimators': [100, 200],          # Số lần lặp của quá trình boosting.
# #     'regressor__learning_rate': [0.01, 0.1],        # Tỉ lệ học
# #     'regressor__max_depth': [3, 6, 10],              # Độ sâu tối đa của cây
# #     'regressor__gamma': [0, 0.1],                   # Mức độ phạt để ngừng phân chia
# #     'regressor__colsample_bytree': [0.7, 0.8, 1.0], # Tỷ lệ cột chọn mẫu cho mỗi cây
# #     'regressor__reg_alpha': [0.1, 1, 10],           # Regularization L1
# #     'regressor__reg_lambda': [0.1, 1, 10]           # Regularization L2
# # }

# param_grid = {
#     'regressor__n_estimators': [50, 100, 150],
#     'regressor__learning_rate': [0.01, 0.05],
#     'regressor__max_depth': [3, 6, 8],
#     'regressor__gamma': [0, 0.1, 0.2],
#     'regressor__colsample_bytree': [0.5, 0.7],
#     'regressor__subsample': [0.7, 0.8],
#     'regressor__reg_alpha': [1, 10, 20],
#     'regressor__reg_lambda': [10, 20, 30]
# }




# # Xây dựng Pipeline với XGBRegressor
# XGB_pipeline = Pipeline(
#     steps=[('preprocessor', preprocessor),  # preprocessor là bước tiền xử lý dữ liệu
#             ('regressor', XGBRegressor())]   # Mô hình XGBRegressor
# )

# # Sử dụng GridSearchCV để tối ưu tham số
# grid_search = GridSearchCV(estimator=XGB_pipeline, param_grid=param_grid, 
#                            cv=15, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# # Huấn luyện GridSearchCV
# grid_search.fit(X_train, y_train)

# # Lấy ra tham số tối ưu và mô hình tối ưu
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# # # Dự đoán và đánh giá mô hình tối ưu
# # y_pred = best_model.predict(X_test)
# # mae = mean_absolute_error(y_test, y_pred)
# # mse = mean_squared_error(y_test, y_pred)
# # r2 = r2_score(y_test, y_pred)

# # # In ra các kết quả đánh giá và tham số tối ưu
# # print("Kết quả đánh giá với tham số tối ưu:")
# # print(f"MAE: {mae}")
# # print(f"MSE: {mse}")
# # print(f"R²: {r2}")

# # Đánh giá trên tập huấn luyện
# y_train_pred = best_model.predict(X_train)
# train_mae = mean_absolute_error(y_train, y_train_pred)
# train_mse = mean_squared_error(y_train, y_train_pred)
# train_r2 = r2_score(y_train, y_train_pred)

# # Đánh giá trên tập kiểm tra
# y_test_pred = best_model.predict(X_test)
# test_mae = mean_absolute_error(y_test, y_test_pred)
# test_mse = mean_squared_error(y_test, y_test_pred)
# test_r2 = r2_score(y_test, y_test_pred)

# # In kết quả so sánh
# print("Hiệu suất trên tập huấn luyện:")
# print(f"MAE (Train): {train_mae}")
# print(f"MSE (Train): {train_mse}")
# print(f"R² (Train): {train_r2}")

# print("\nHiệu suất trên tập kiểm tra:")
# print(f"MAE (Test): {test_mae}")
# print(f"MSE (Test): {test_mse}")
# print(f"R² (Test): {test_r2}")

# # So sánh để xác định overfitting
# if train_r2 - test_r2 > 0.1:  # Chênh lệch R² lớn hơn 0.1 là dấu hiệu overfitting
#     print("\nCẢNH BÁO: Mô hình có dấu hiệu overfitting!")
# else:
#     print("\nMô hình không có dấu hiệu overfitting.")

# # In ra các tham số tối ưu
# print("\nCác tham số tối ưu của XGBRegressor:")
# for param, value in best_params.items():
#     print(f"{param}: {value}")


Fitting 15 folds for each of 1944 candidates, totalling 29160 fits


KeyboardInterrupt: 